# Actividad RGPD: Anonimización e integración de datos



Trabajaremos con:



- `rgpd_practica/clientes_raw.csv`

- `rgpd_practica/ventas_clientes.csv`



Objetivos:



1. Identificar datos personales y columnas sensibles.

2. Aplicar anonimización (hash, eliminación, generalización).

3. Generar `clientes_anon.csv`.

4. Integrar las ventas con el identificador anónimo.

5. Aplicar reglas de calidad sobre el dataset integrado.


In [3]:
import pandas as pd

from pathlib import Path

import hashlib



base_path = Path("rgpd_practica")



clientes = pd.read_csv(base_path / "clientes_raw.csv")

ventas = pd.read_csv(base_path / "ventas_clientes.csv")



clientes.head(), ventas.head()

(   id_cliente   nombre  apellidos                          email   telefono  \
 0           1     Luis      López       luis.lópez_1@example.com  643321819   
 1           2   Manuel      López     manuel.lópez_2@example.com  683863794   
 2           3    Elena  Rodríguez  elena.rodríguez_3@example.com  616155940   
 3           4   Manuel    Álvarez   manuel.álvarez_4@example.com  693103413   
 4           5  Alberto    Álvarez  alberto.álvarez_5@example.com  634192832   
 
   fecha_nacimiento    ciudad  cod_postal  gasto_mensual  frecuencia_compra  \
 0       1997-11-07    Madrid       28001          74.34                  3   
 1       1960-07-31  Valencia       46023         217.35                  2   
 2       2001-03-20   Sevilla       41010          65.70                  4   
 3       1969-01-22    Murcia       30002         282.98                  5   
 4       2001-06-21    Murcia       30002         593.92                 10   
 
   ultima_compra  
 0    2024-12-27  
 1  

## 1. Identificación de datos personales



En tu informe (fuera del notebook) clasifica las columnas de `clientes_raw.csv` en:



- Datos personales directos

- Datos personales indirectos

- Datos no personales



Explica brevemente por qué.

In [4]:
def make_hash(x: int) -> str:

    return hashlib.sha256(str(x).encode()).hexdigest()



clientes["id_hash"] = clientes["id_cliente"].apply(make_hash)

clientes[["id_cliente", "id_hash"]].head()

,id_cliente,id_hash
0,1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...
1,2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...
2,3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...
3,4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...
4,5,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...


In [5]:
# Generalizar código postal a los 3 primeros dígitos

clientes["cp_trunc"] = clientes["cod_postal"].astype(str).str[:3]



# (Opcional) generalizar fecha de nacimiento a década

clientes["decada_nacimiento"] = clientes["fecha_nacimiento"].str[:3] + "0s"



# Dataset anonimizado: elimina columnas directas

cols_drop = ["nombre", "apellidos", "email", "telefono", "cod_postal", "fecha_nacimiento"]

clientes_anon = clientes.drop(columns=cols_drop)



clientes_anon.head()

,id_cliente,ciudad,gasto_mensual,frecuencia_compra,ultima_compra,id_hash,cp_trunc,decada_nacimiento
0,1,Madrid,74.34,3,2024-12-27,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,280,1990s
1,2,Valencia,217.35,2,2025-02-02,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,460,1960s
2,3,Sevilla,65.70,4,2024-12-11,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,410,2000s
3,4,Murcia,282.98,5,2025-02-09,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,300,1960s
4,5,Murcia,593.92,10,2024-12-03,ef2d127de37b942baad06145e54b0c619a1f22327b2ebb...,300,2000s


In [6]:
ventas_hash = ventas.merge(

    clientes_anon[["id_cliente", "id_hash", "ciudad", "cp_trunc", "decada_nacimiento"]],

    on="id_cliente",

    how="left"

)

# Eliminar id_cliente en la tabla final (ya tenemos id_hash)
ventas_hash = ventas_hash.drop(columns=["id_cliente"])

ventas_hash.head()

,id_venta,fecha,unidades,importe,canal,ciudad_x,cod_postal,municipio_id,id_hash,ciudad_y,cp_trunc,decada_nacimiento
0,1,2025-02-22,7,5.35,app,Sevilla,41005,410,27d719c754aacd492a6dc8a1b76619355abcf5ef473cbe...,Sevilla,410,1990s
1,2,2025-01-24,9,263.07,web,Zaragoza,50001,500,0f4121d0ef1df4c86854c7ebb47ae1c93de8aec8f94403...,Zaragoza,500,1960s
2,3,2025-02-21,7,147.23,web,Valladolid,47010,470,85daaf6f7055cd5736287faed9603d712920092c4f8fd0...,Valladolid,470,1990s
3,4,2025-01-07,5,130.52,web,Málaga,29001,290,9cfd3c755be26b4e1645918e2a64a26e3d851ede421e0b...,Málaga,290,1970s
4,5,2025-02-28,5,258.42,app,Valencia,46002,460,2abaca4911e68fa9bfbf3482ee797fd5b9045b841fdff7...,Valencia,460,1980s


In [7]:
total = len(ventas_hash)

dom_canal = (ventas_hash["canal"].isin(["tienda", "web", "app"])).mean()

rango_unidades = (ventas_hash["unidades"] >= 0).mean()

importe_pos = (ventas_hash["importe"] > 0).mean()



print(f"Total registros: {total}")

print(f"Dominio canal correcto: {dom_canal:.2%}")

print(f"Unidades >= 0: {rango_unidades:.2%}")

print(f"Importe > 0: {importe_pos:.2%}")

Total registros: 1000
Dominio canal correcto: 100.00%
Unidades >= 0: 100.00%
Importe > 0: 100.00%


In [8]:
clientes_anon_path = base_path / "clientes_anon.csv"

ventas_hash_path = base_path / "ventas_clientes_anon.csv"



clientes_anon.to_csv(clientes_anon_path, index=False)

ventas_hash.to_csv(ventas_hash_path, index=False)



clientes_anon_path, ventas_hash_path

(WindowsPath('rgpd_practica/clientes_anon.csv'),
 WindowsPath('rgpd_practica/ventas_clientes_anon.csv'))